# Limpeza de dados

Este notebook tem como objetivo executar as rotinas de limpeza e tratamento de dados definidas a partir das inconsistências identificadas na etapa de Análise Exploratória.

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
BASE_PATH = Path("../../../")
DATA_RAW = BASE_PATH / "data" / "raw"
DATA_INTERIM = BASE_PATH / "data" / "interim" / "T3" / "regressao"

orders = pd.read_csv(DATA_RAW / "olist_orders_dataset.csv")
products = pd.read_csv(DATA_RAW / "olist_products_dataset.csv")
geolocation = pd.read_csv(DATA_RAW / "olist_geolocation_dataset.csv")

## Status do Pedido (`order_status`)

In [3]:
print(f"Antes: {orders.shape[0]} registros")

# Mantém apenas pedidos entregues
orders_clean = orders[orders['order_status'] == 'delivered'].copy()

print(f"Depois: {orders_clean.shape[0]} registros")

Antes: 99441 registros
Depois: 96478 registros


## Dimensões e Peso dos Produtos (`product_weight_g`, `product_length_cm`, `product_height_cm`, `product_width_cm`)

In [4]:
print(f"Antes: {products.shape[0]} registros")

cols_dimensao = ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

# Remove valores nulos
products_clean = products.dropna(subset=cols_dimensao)

# Mantém apenas valores acima de 0
condicao_maior_que_zero = (
    (products_clean['product_weight_g'] > 0) &
    (products_clean['product_length_cm'] > 0) &
    (products_clean['product_height_cm'] > 0) &
    (products_clean['product_width_cm'] > 0)
)
products_clean = products_clean[condicao_maior_que_zero].copy()

print(f"Depois: {products_clean.shape[0]} registros")

Antes: 32951 registros
Depois: 32945 registros


## Geolocalização (`geolocation_lat`, `geolocation_lng`)

In [5]:
print(f"Antes: {geolocation.shape[0]} registros")

lat_norte = 5 + 16/60 + 19/3600
lat_sul = -(33 + 45/60 + 9/3600)
lon_leste = -(34 + 45/60 + 54/3600)
lon_oeste = -(73 + 59/60 + 32/3600)

# Mantém apenas coordenadas dentro dos limites territoriais do Brasil
geo_clean = geolocation[
    (geolocation["geolocation_lat"] >= lat_sul) &
    (geolocation["geolocation_lat"] <= lat_norte) &
    (geolocation["geolocation_lng"] >= lon_oeste) &
    (geolocation["geolocation_lng"] <= lon_leste)
].copy()

# Agrupa registros por CEP e preenche coordenadas pela média.
geo_agg = geo_clean.groupby("geolocation_zip_code_prefix", as_index=False).agg({
    "geolocation_lat": "mean",
    "geolocation_lng": "mean",
    "geolocation_city": "first",
    "geolocation_state": "first"
})

print(f"Depois: {geo_agg.shape[0]} registros")

Antes: 1000163 registros
Depois: 19010 registros


## Exportação dos Dados

In [6]:
orders_clean.to_csv(DATA_INTERIM / "orders_clean.csv", index=False)
products_clean.to_csv(DATA_INTERIM / "products_clean.csv", index=False)
geo_agg.to_csv(DATA_INTERIM / "geolocation_agg.csv", index=False)